In [14]:
# From pipeline textbook chapter, adapted for full accident control data
import pandas as pd


In [15]:
from sklearn.preprocessing import LabelEncoder
X_train = pd.read_csv("../Data_Cleaned/train_data/control_2class_X_train.csv")
y_train = pd.read_csv("../Data_Cleaned/train_data/control_2class_y_train.csv")
X_test = pd.read_csv("../Data_Cleaned/test_data/control_2class_X_test.csv")
y_test = pd.read_csv("../Data_Cleaned/test_data/control_2class_y_test.csv")

y_train = y_train.values.ravel()
y_test = y_test.values.ravel()

le = LabelEncoder()
y_train = le.fit_transform(y_train)
le.classes_

array([0, 1])

In [16]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline

# NEED TO FIGURE OUT ONE HOT ENCODER FOR PIPELINE, AND ADD IN FIRST
# Chose model based on https://scikit-learn.org/stable/machine_learning_map.html
# NEED TO ADD ARGS TO ALL THESE ASAP, AND CONFIRM I AM FOLLOWING HOW IT WORKS PROPERLY

pipe_lr = make_pipeline(OneHotEncoder(),
                        StandardScaler(with_mean=False),
                        PCA(n_components=2,svd_solver='arpack'),
                        LinearSVC())

pipe_lr.fit(X_train, y_train)
y_pred = pipe_lr.predict(X_test)
test_acc = pipe_lr.score(X_test, y_test)
print(f"Test Accuracy: {test_acc:.3f}")

ValueError: Found unknown categories [np.float64(20.8), np.float64(23.7), np.float64(63.3), np.float64(89.2), np.float64(89.8), np.float64(90.3), np.float64(94.3), np.float64(96.1)] in column 1 during transform

In [ ]:
from sklearn.model_selection import cross_val_score
import numpy as np
from sklearn.model_selection import StratifiedKFold

scores = cross_val_score(estimator = pipe_lr,
                         X = X_train,
                         y = y_train,
                         cv = 10,
                         n_jobs = 1)
print(f'CV Accuracy scores: {scores}')
print(f'CV Accuracy: {np.mean(scores):.3f} '
      f'+/- {np.std(scores):.3f}')

CV Accuracy scores: [0.78606295 0.78619529 0.78619529 0.78619529 0.78619529 0.78619529
 0.78619529 0.78619529 0.78602694 0.78602694]
CV Accuracy: 0.786 +/- 0.000


In [6]:
import numpy as np
from sklearn.model_selection import StratifiedKFold

kfold = StratifiedKFold(n_splits = 10).split(X_train, y_train)
scores = []
for k, (train, test) in enumerate(kfold):
    pipe_lr.fit(X_train[train], y_train[train])
    score = pipe_lr.score(X_train[test], y_train[test])
    scores.append(score)
    print(f'Fold: {k+1:02d}, '
          f'Class distr.: {np.bincount(y_train[train])}, '
          f'Acc.: {score:.3f}')
mean_acc = np.mean(scores)
std_acc = np.std(scores)
print(f'\nCV accuracy: {mean_acc:.3f} +/- {std_acc:.3f}')

Fold: 01, Class distr.: [  157 42028  4808  6467], Acc.: 0.786
Fold: 02, Class distr.: [  158 42028  4808  6467], Acc.: 0.786
Fold: 03, Class distr.: [  158 42028  4808  6467], Acc.: 0.786
Fold: 04, Class distr.: [  158 42028  4808  6467], Acc.: 0.786
Fold: 05, Class distr.: [  158 42028  4807  6468], Acc.: 0.786
Fold: 06, Class distr.: [  158 42028  4807  6468], Acc.: 0.786
Fold: 07, Class distr.: [  157 42028  4808  6468], Acc.: 0.786
Fold: 08, Class distr.: [  157 42028  4808  6468], Acc.: 0.786
Fold: 09, Class distr.: [  157 42029  4808  6467], Acc.: 0.786
Fold: 10, Class distr.: [  157 42029  4808  6467], Acc.: 0.786

CV accuracy: 0.786 +/- 0.000
